In [1]:
import numpy as np
from skyfield.api import load,Topos
from skyfield.earthlib import reverse_terra,terra

stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle(stations_url)
ts = load.timescale(builtin=True)

au_to_km = 149598000

In [2]:
t = ts.now()
g_lat = 20
g_lon = 10
g = Topos(latitude_degrees=g_lat, longitude_degrees=g_lon, elevation_m=0)

diff = 70
viewable = {}
for name in satellites.keys():
    if not isinstance(name,int):
        continue
    sat = satellites[name]
    s   = sat.at(t).subpoint()
    if  np.abs(s.latitude.degrees - g.latitude.degrees) < diff and np.abs(s.longitude.degrees - g.longitude.degrees) < diff:
        viewable[name] = sat

print("%d/%d" % (len(viewable.keys()), len(satellites.keys())))


21/156


In [3]:
import random

sats = random.choices(list(viewable.keys()), k=5)
Vs = list(map(lambda s: viewable[s].at(t).position.au, sats))
Vg = g.at(t).position.au


Vg,_ = terra(np.deg2rad(g.latitude.degrees), np.deg2rad(g.longitude.degrees), 0, t.gast)

for V in Vs:
    print(V)
print(Vg)

[-2.27560365e-05  1.64228044e-05  3.51381157e-05]
[ 2.38098328e-06  4.41470367e-05 -9.74893498e-06]
[-1.69586817e-05  3.08361640e-05  2.82665326e-05]
[-2.27560365e-05  1.64228044e-05  3.51381157e-05]
[ 4.49198700e-07  4.24133870e-05 -1.25542072e-05]
[-2.79616432e-05  2.87145941e-05  1.44901569e-05]


In [4]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    Reference:
        https://stackoverflow.com/a/29546836/7657658
    
    https://gist.github.com/mazzma12/6dbcc71ab3b579c08d66a968ff509901
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(
        dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

while True:
    t_lat = g_lat + np.random.rand() - 0.5
    t_lon = g_lon + np.random.rand() - 0.5
    diff = haversine_np(t_lon, t_lat, g_lon, g_lat) 
    if diff < 20 and diff > 10:
        break
Target = Topos(latitude_degrees=t_lat, longitude_degrees=t_lon, elevation_m=0)
tpos,_ = terra(np.deg2rad(t_lat), np.deg2rad(t_lon), 0, t.gast)
print(Target)
print(tpos)

Topos 19deg 52' 41.6" N 09deg 54' 52.9" E
[-2.79402659e-05  2.87782438e-05  1.44054424e-05]


In [5]:
Rg = np.linalg.norm(Vg - tpos)
print(Rg)
Rs = list(map(lambda V : np.linalg.norm(V - tpos) - Rg, Vs))
Rg = 0
print(Rg)
for R in Rs:
    print(R)


1.0809628981073418e-07
0
2.4577463414904668e-05
4.1593365247075314e-05
1.7695275557137735e-05
2.4577463414904668e-05
4.134912913338551e-05


In [6]:
def F(Vi, Vj):
    return  (Vi[0]**2 - Vj[0]**2) + (Vi[1]**2 - Vj[1]**2) + (Vi[2]**2 - Vj[2]**2)

d = 1 / au_to_km
bestDiff = 10000000000
bestD = d
print(bestD)
while d < (25 / au_to_km):
    rows = int( (len(Rs) + 1) * len(Rs) / 2)
    A = np.zeros((rows, 3))
    C = np.zeros(rows)

    kk = 0
    for ii in range(0,len(Rs)-1):
        A[kk,:] = [ 2*(Vs[ii][0] - Vg[0]), 2*(Vs[ii][1] - Vg[1]), 2*(Vs[ii][2] - Vg[2]) ]
        C[kk] = F(Vs[ii],Vg) - ( (d+Rs[ii])**2 - (d+Rg)**2)
        kk += 1
        
        for jj in range(ii+1,len(Rs)):
            A[kk,:] = [ 2*(Vs[ii][0] - Vs[jj][0]), 2*(Vs[ii][1] - Vs[jj][1]), 2*(Vs[ii][2] - Vs[jj][2]) ]
            C[kk] = F(Vs[ii],Vs[jj]) - ( (d+Rs[ii])**2 - (d+Rs[jj])**2)
            kk += 1

    guess = np.linalg.pinv(A) @ C.T
    diff = np.abs(np.linalg.norm(guess) - np.linalg.norm(Vg))
    if diff < bestDiff:
        bestGuess = guess
        bestDiff = diff
        bestD = d
    d += 0.001 / au_to_km
print(bestD)
print(bestDiff)

6.684581344670383e-09
1.0800946536715537e-07
1.1889895670672349e-13


In [7]:
guess = np.linalg.pinv(A) @ C.T
#guess = guess * (np.linalg.norm(Vg)/np.linalg.norm(guess))
print(tpos)
print(guess.T)
print(np.linalg.norm(guess.T - tpos))
print(np.linalg.norm(guess), np.linalg.norm(Vg))

[-2.79402659e-05  2.87782438e-05  1.44054424e-05]
[-2.80565360e-05  2.88568766e-05  1.44117891e-05]
1.4050662669006786e-07
4.275030061541313e-05 4.2618611581557584e-05


In [8]:
guess = bestGuess
lat, lon, elevation = reverse_terra(guess, t.gast, iterations=1000)
print("{}, {}, {}".format(np.rad2deg(lat),np.rad2deg(lon), elevation ))
print("{}, {}".format(Target.latitude.degrees, Target.longitude.degrees))

19.878303267626073, 9.914631562540494, -28.91735353138757
19.878222238855052, 9.91469176934299


In [9]:


print(1000*haversine_np(t_lat, t_lon, g_lat, g_lon))
print(1000*haversine_np(np.rad2deg(lon), np.rad2deg(lat), Target.longitude.degrees, Target.latitude.degrees))

16366.408245057144
10.991679471418246
